In [7]:
import matplotlib.pyplot as plt
from Code_utilities.FromGlazPPAnalysis.Glaz_PP_Series_analysis import GlazPPSeriesAnalysis
from Code_utilities.FromGlazPPAnalysis.GlazPumpProbeDataClass import GlazPumpProbeData

plt.close('all')
parent_dir_path ='/Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS'
GlazPPSeriesAnalysis = GlazPPSeriesAnalysis(parent_dir_path, )

GlazPPSeriesAnalysis.plot_all_spectrum_timetraces(keyword='SpectralDifference',
                                                  bool_plot_normalized_time_trace=False)

Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_093759_woPump_0.txt
DSTMS LP 10 THz
Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_102132_woPump_0.txt
DSTMS LP 3+2 THz
Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_124150_woPump_0.txt
PNPA LP 3+1 THz, longer range
Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_095549_woPump_0.txt
DSTMS LP 3 THz


(<Figure size 1000x600 with 2 Axes>,
 <Axes: title={'center': 'Differential Spectrum_DSTMS LP 3 THz'}, xlabel='Pixel', ylabel='Delay (ps)'>)

In [8]:
import pandas as pd
import numpy as np

def spectral_centroid_and_spread(
    frequency_axis_thz,
    power_spectrum,
    lower_freq_thz: float = 0.0,
    upper_freq_thz: float | None = None
):
    """
    Compute power-weighted spectral centroid and spread over a chosen band.
    Works with pandas Series or numpy arrays. Drops NaNs and sorts by frequency.
    """
    # Convert to numpy arrays
    f = frequency_axis_thz.to_numpy() if isinstance(frequency_axis_thz, pd.Series) else np.asarray(frequency_axis_thz)
    w = power_spectrum.to_numpy() if isinstance(power_spectrum, pd.Series) else np.asarray(power_spectrum)

    # Remove NaNs and ensure same length
    valid = np.isfinite(f) & np.isfinite(w)
    f, w = f[valid], w[valid]
    if f.size == 0:
        return np.nan, np.nan

    # Sort by frequency in case it's not strictly increasing
    order = np.argsort(f)
    f, w = f[order], w[order]

    if upper_freq_thz is None:
        upper_freq_thz = f[-1]  # positional last element

    # In-band mask
    in_band = (f >= lower_freq_thz) & (f <= upper_freq_thz)
    f_band = f[in_band]
    w_band = w[in_band]
    if f_band.size == 0:
        return np.nan, np.nan

    # Integrals
    total_power = np.trapz(w_band, f_band)
    if total_power <= 0:
        return np.nan, np.nan

    centroid = np.trapz(f_band * w_band, f_band) / total_power
    spread  = (np.trapz(w_band, f_band))**2 / np.trapz(w_band**2, f_band)
    return centroid, spread

fig_ffts, ax_ffts, data_dict = GlazPPSeriesAnalysis.plot_all_ffts_together(pixel=310,
                                                                           window_start=0,
                                                                           window_end=6,
                                                                           bool_organize_legend=True,
                                                                           bool_save_fig=True,
                                                                           plot_x_lim_min=0,
                                                                           plot_x_lim_max=10,
                                                                           bool_return_data=True, )
weighted_average_df = pd.DataFrame(columns=['label',
                                            'centroid',
                                            'spread'])

for label, df in data_dict.items():
    df['FFT**2'] = df['FT Magnitude'] ** 2
    centroid, spread=spectral_centroid_and_spread(frequency_axis_thz=df['Frequency (THz)'],
                                                  power_spectrum=df['FFT**2'],)
    weighted_average_df.loc[len(weighted_average_df)] = {
    'label': label,
    'centroid': centroid,
    'spread': spread
}



# fig_ffts.show()
#
# data_df=GlazPPSeriesAnalysis.plot_all_timetraces_at_pixel_separately_returning_data(pixel = 310,
#                                                              number_of_pixels_for_average=5,
#                                                              spectrum_type_keyword='SpectralDifference',
#                                                              save_fig_bool = True)
#
# file_path=data = GlazPumpProbeData_v2('/Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_093759_SpectralDifference_0.txt')
#
# data = GlazPumpProbeData(file_path,
#                          298.498477,
#                          303.090665,
#                          0.01)
# data.plot_spectral_timetrace()
# data.plot_cross_section(600)
# data.plot_fft(600, 301.5, 303)




Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_093759_woPump_0.txt
Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_102132_woPump_0.txt
Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_124150_woPump_0.txt
Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_095549_woPump_0.txt
Found unpumped file: /Users/Shared/Files From c.localized/Gabriel_UniBern_Local/DataAnalysis/Low cost THz Camera/SpectralResponseComparison/EOS/20250618_093759_woPump_0.txt


TypeError: GlazPumpProbeData.__init__() takes from 2 to 3 positional arguments but 5 were given